## MCQ and Answeres 

In [1]:
import pandas as pd
import sqlite3

In [2]:
orders = pd.read_csv("orders.csv")
orders

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian
...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian


In [7]:
users = pd.read_json("users.json")
users

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


In [32]:
conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as f:
    sql_script = f.read()
conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8
...,...,...,...,...
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9


In [34]:
final_df = orders.merge(users, on="user_id", how="left") \
                 .merge(restaurants, on="restaurant_id", how="left")

In [36]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["order_date"]

0      2023-02-18
1      2023-01-18
2      2023-07-15
3      2023-10-04
4      2023-12-25
          ...    
9995   2023-05-21
9996   2023-08-06
9997   2023-11-11
9998   2023-09-08
9999   2023-10-21
Name: order_date, Length: 10000, dtype: datetime64[ns]

In [42]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)
print("Final dataset rows:", final_df.shape[0])

Final dataset rows: 10000


# MCQ'S

**1. Which city has the highest total revenue (total_amount) from Gold members?** 

In [44]:
a = final_df[final_df["membership"]=="Gold"] \
        .groupby("city")["total_amount"].sum().idxmax()
a

'Chennai'

**2. Which cuisine has the highest average order value across all orders?**

In [46]:
b = final_df.groupby("cuisine")["total_amount"].mean().idxmax()
b

'Mexican'

**3. How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?**

In [48]:
user_totals = final_df.groupby("user_id")["total_amount"].sum()
c = (user_totals > 1000).sum()
c

2544

**4. Which restaurant rating range generated the highest total revenue?**

In [52]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [54]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[0, 2, 3, 4, 5],
    labels=["0-2", "2-3", "3-4", "4-5"],
    include_lowest=True
)


In [58]:
d = final_df.groupby(
    "rating_range", observed=False
)["total_amount"].sum().idxmax()
d

'4-5'

**5. Among Gold members, which city has the highest average order value?**

In [60]:
e = final_df[final_df["membership"]=="Gold"] \
        .groupby("city")["total_amount"].mean().idxmax()
e

'Chennai'

**6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenu?**

In [62]:
f = final_df.groupby("cuisine")["restaurant_id"].nunique().idxmin()
f

'Chinese'

**7. What percentage of total orders were placed by Gold members? (Rounded to nearest integer)**

In [64]:
g = round((final_df["membership"]=="Gold").mean()*100)
g

50

**8. Which restaurant has the highest average order value but less than 20 total orders?**

In [66]:
print(final_df.columns)

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range'],
      dtype='object')


In [68]:
# Automatically pick the correct restaurant column
if "restaurant_name_y" in final_df.columns:
    restaurant_col = "restaurant_name_y"
elif "restaurant_name" in final_df.columns:
    restaurant_col = "restaurant_name"
else:
    raise Exception("Restaurant name column not found!")


In [70]:
# Group by restaurant
rest_stats = final_df.groupby(restaurant_col).agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

# Filter < 20 orders
filtered = rest_stats[rest_stats["total_orders"] < 20]

# Sort by highest average order value
best_restaurant = filtered.sort_values(
    "avg_order_value", ascending=False
)

print(best_restaurant.head(10))   # DEBUG: see top 10
print("\nFinal Answer:", best_restaurant.index[0])


                   avg_order_value  total_orders
restaurant_name_y                               
Restaurant_294         1040.222308            13
Restaurant_262         1029.473333            18
Restaurant_77          1029.180833            12
Restaurant_193         1026.306667            15
Restaurant_7           1002.140625            16
Restaurant_298          989.822000            15
Restaurant_56           989.467222            18
Restaurant_135          988.702222            18
Restaurant_343          986.026471            17
Restaurant_312          982.730000            14

Final Answer: Restaurant_294


**9. Which combination contributes the highest revenue?**

In [74]:
h = final_df.groupby(["membership","cuisine"])["total_amount"].sum().idxmax()
h

('Regular', 'Mexican')

**10. During which quarter of the year is the total revenue highest?**

In [76]:
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")
i = final_df.groupby("quarter")["total_amount"].sum().idxmax()
i

Period('2023Q3', 'Q-DEC')

**11. How many total orders were placed by users with Gold membership?**

In [78]:
gold_orders = (final_df["membership"]=="Gold").sum()
gold_orders

4987

**12. What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?**

In [80]:
Hyderabad_revenue = round(
    final_df[final_df["city"]=="Hyderabad"]["total_amount"].sum())
Hyderabad_revenue

1889367

**13. How many distinct users placed at least one order?**

In [82]:
distinct_users = final_df["user_id"].nunique()
distinct_users

2883

**14. What is the average order value (rounded to 2 decimals) for Gold members?**

In [84]:
gold_aov = round(
    final_df[final_df["membership"]=="Gold"]["total_amount"].mean(),2)
gold_aov

797.15

**15. How many orders were placed for restaurants with rating ≥ 4.5?**

In [86]:
high_rating_orders = (final_df["rating"]>=4.5).sum()
high_rating_orders

3374

**16.How many orders were placed in the top revenue city among Gold members only?**

In [88]:
top_gold_city = final_df[final_df["membership"]=="Gold"] \
                    .groupby("city")["total_amount"].sum().idxmax()

gold_orders_top_city = len(
    final_df[
        (final_df["membership"]=="Gold") &
        (final_df["city"]==top_gold_city)
    ]
)
gold_orders_top_city

1337